In [27]:
import numpy as np
import math
import matplotlib.pyplot as plt

# from grad import norm_of_surface
from vector import Vec
# from intercept import within_error, estimate_all_intercepts, nearest_point, is_in_interval

In [28]:
# estimate_all_intercepts(f, g, interval, n_tests = 100, n_iter = 100, error = 1e-10):

N_TESTS = 1000
N_ITER = 100
ERROR = 1e-10
CLIP_DIST = 1e-5

In [29]:
def sign(num):
    val = np.sign(num)
    if val != 0:
        return val
    else:
        return 1

    
def hit(inc: Vec, norm: Vec, n1, n2) -> list[Ray]:
    
    reflected_direction = reflect(inc, norm)
    refracted_direction = refract(inc, norm, n1, n2)
    
    
    if refracted_direction is None:
        # Only reflection
        
        
        return ...
    
    cos_i = inc ^ norm
    cos_r = reflected_direction ^ norm
    cos_t = refracted_direction ^ norm
    
    z1_cos_i = n1 * cos_i
    z1_cos_t = n1 * cos_t
    
    z2_cos_i = n2 * cos_i
    z2_cos_t = n2 * cos_t
    
    # Since we are using vectors, the incident and refracted are the same sign when 
    # dotted with the norm, so we do not need to worry about signs
    
    # s polarized
    Rs = ((z2_cos_i - z1_cos_t) / (z2_cos_i + z1_cos_t)) ** 2
    # p polarized
    Rp = ((z2_cos_t - z1_cos_i) / (z2_cos_t + z1_cos_i)) ** 2
    
    R = (Rs + Rp) * 0.5
    T = 1 - R
    

def reflect(inc: Vec, norm: Vec) -> Vec:
    if not isinstance(inc, Vec):
        raise TypeError("inc must be a Vec type")
    if not isinstance(norm, Vec):
        raise TypeError("norm must be a Vec type")
    
    inc.normalise()
    norm.normalise()
    
    A = inc ^ norm
    
    reflected_direction = (inc - 2 * A).return_normalised()
    return reflected_direction

def refract(inc: Vec, norm: Vec, n1, n2) -> Vec:
    if not isinstance(inc, Vec):
        raise TypeError("inc must be a Vec type")
    if not isinstance(norm, Vec):
        raise TypeError("norm must be a Vec type")
    
    inc.normalise()
    norm.normalise()
    
    A = inc ^ norm
    A2 = A * A
    mu = n1 / n2
    mu2 = mu * mu
    
    # Cannot sqrt negative numbers
    if mu2 * (1 - A2) > 1:
        return None
    
    cp1 = norm * (sign(A) * (1 - mu2 * (1 - A2)) ** 0.5 - (A * mu))
    cp2 = mu * inc
    
    refracted_direction = (cp1 + cp2).return_normalised()
    
    return refracted_direction


In [30]:
class Ray:
    def __init__(self, direction: Vec, start_pos: Vec, start_boundary = None, rel_intensity = 1, level = 0):
        
        if not isinstance(direction, Vec):
            raise TypeError("direction must be a Vec type")
        if not isinstance(start_pos, Vec):
            raise TypeError("start_pos must be a Vec type")
        
        self.direction = direction
        self.direction.normalise()
        
        self.start_pos = start_pos
        
        # If start_boundary = None, then treat the ray to be in air (n = 1)
        self.start_boundary = start_boundary
        
        self.rel_intensity = rel_intensity
        
        # The number of interactions have happened before this ray's existance
        # Very first ray is level = 0, reflected or refracted is level = 1, next interaction is level = 2
        self.level = level
        
        self.end_boundary = None
        self.end_pos = None
        self.refracted_ray = None
        self.reflected_ray = None
        
    def __str__(self):
        
        result = {
            "class": self.__class__.__name__,
            "direction": self.direction,
            "start_pos": self.start_pos,
            "end_pos": self.end_pos,
            "start_boundary": self.start_boundary,
            "end_boundary": self.end_boundary,
            "level": self.level,
            "refracted_ray": self.refracted_ray,
            "reflected_ray": self.reflected_ray,
            
        }
        
        prt_str = "\n"
        for key, val in result.items():
            for i in range(self.level):
                prt_str+="-"
            prt_str+=f"{key}: {val}\n"
         
        return prt_str
        
    # Finds distance from start position given 1 coordinate from one of x, y or z
    def where(self, x = None, y = None, z = None) -> float:
        
        where_pos = np.array([x, y, z])
        val_arg = 0
        
        None_count = 0
        for i, val in enumerate(where_pos):
            if val is None:
                None_count+=1
            else:
                val_arg = i
                
        if None_count != 2:
            raise ValueError(f"Number of arguments must equal 1, but got {3 - None_count}")
            
            
        # Only 1 value to test
        if self.direction[val_arg] == 0:
            return None
        
        dist = (where_pos[val_arg] - self.start_pos[val_arg]) / self.direction[val_arg]
        
        # Cant have points before the starting position
        if dist < 0:
            return None
        
        return dist
        
        
    def pos_from_distance(self, dist: float) -> Vec:
        return self.start_pos + dist * self.direction
    
    def pos_from_one_coord(self, **kwargs) -> Vec:
        dist = self.where(**kwargs)
        
        if dist is None:
            return None
        else:
            return self.pos_from_distance(dist)
        
    
    def update_ray(self, clipping_distance: float = CLIP_DIST) -> bool:
        
        # Must have an end boundary (need to know norm and refractive index)
        if self.end_boundary is None:
            return False
        
        self.end_pos = self.pos_from_distance(self.end_boundary.ray_intercept_dist(self))
        
        # Must have an end position (None suggests it goes to inf)
        if self.end_pos is None:
            return False
        
        
        
        # Add a very small distance to stop ray clipping to boundaries
        refracted_pos = self.end_pos + clipping_distance * self.direction
        reflected_pos = self.end_pos - clipping_distance * self.direction
        
        # Initial refractive index
        n1 = 1
        if self.start_boundary is None:
            n1 = 1
        else:
            n1 = self.start_boundary.get_n()
        
        # The refractive index of the new medium 
        n2 = self.end_boundary.get_n()
        
        reflected_direction = reflect(
            self.direction, 
            self.end_boundary.get_norm(self.end_pos)
        )
        
        refracted_direction = refract(
            self.direction, 
            self.end_boundary.get_norm(self.end_pos),
            n1,
            n2
        )
        
        if reflected_direction is not None:
            self.reflected_ray = Ray( 
                reflected_direction,
                Vec(reflected_pos),
                # The ray is in the same medium
                start_boundary = self.start_boundary,
                level = self.level+1
            )
        
        if refracted_direction is not None:
            self.refracted_ray = Ray( 
                refracted_direction,
                Vec(refracted_pos),
                # The ray has entered the new medium
                start_boundary = self.end_boundary,
                level = self.level+1
            )
        
        return True

In [31]:
class Plane:
    def __init__(self, norm: Vec, pos: Vec):
        if not isinstance(norm, Vec):
            raise TypeError("norm must be a Vec type")
        if not isinstance(pos, Vec):
            raise TypeError("pos must be a Vec type")
        
        self.norm = norm
        self.norm.normalise()
        
        self.p0 = pos
        
        self.parent = None
        
    def __str__(self):
        return f"{self.__class__.__name__}: (norm = {self.norm}, p0 = {self.p0})"
        
    def ray_intercept_dist(self, ray: Ray, include_negative: bool = False, parallel_case: bool = False) -> float:
        if not isinstance(ray, Ray):
            raise TypeError("ray must be a Ray type")
        
        # d = n_p * (r_plane0 - r_ray0) / (n_plane * n_ray)
        
        dot = ray.direction ^ self.norm
        
        if abs(dot) < ERROR:
            # Ray is parallel
            if not parallel_case:
                return None
            else:
                return self.point_is_in(ray.start_pos)
        
        top_part = self.norm ^ (self.p0 - ray.start_pos)
        
        d = top_part / dot
        
        
        if d < 0 and not include_negative:
            # Ray has passed plane already
            return None
        else:
            return d
        
    def point_is_in(self, point: Vec) -> bool:
        if not isinstance(point, Vec):
            raise TypeError("point must be a Vec type")
        return ((self.p0 - point) ^ self.norm) < 0
    
    def get_n(self):
        return self.parent.n
    
    def get_norm(self, *args, **kwargs):
        return self.norm
        
    
     
class PlaneSurface:
    def __init__(self, *surfaces, n = 1):
        
        # Refractive index
        self.n = n
        self.surfaces = surfaces
        
        for surface in self.surfaces:
            surface.parent = self
        
    
    def __len__(self):
        return len(self.surfaces)
    
    def __bool__(self):
        return len(self) > 0
        
    def __getitem__(self, i):
        return self.surfaces[i]
    
    def __setitem__(self, i, val):
        self.surfaces[i] = val
        
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        
        if self.index < len(self):
            val = self.surfaces[self.index]
            self.index += 1
            return val
        raise StopIteration
        
    def find_intercept(self, ray: Ray):
        if not isinstance(ray, Ray):
            raise TypeError("ray must be a Ray type")
        
        surface_dist_intercept = []
        index = []
        
        for i, surface in enumerate(self):
            d = surface.ray_intercept_dist(ray, include_negative = True, parallel_case = True)
#             print(i, d, ray, surface)
            if d is None or d == False:
                return None
            elif d != True:
                # Never need to check plane if ray is parrallel and inside to plane
                index.append(i)
                surface_dist_intercept.append( d )
            
        surface_dist_intercept = np.array(surface_dist_intercept)
        index = np.array(index)
        
        
        argsort_dist = np.argsort(surface_dist_intercept)
        
        sorted_index = index[argsort_dist]
        
        sorted_dist = surface_dist_intercept[argsort_dist]
        
        
        # check which side ray lies at -inf
        # can take arbitary value away, choose 1
        d_inf = sorted_dist[0] - 1
        
        ray_pos_inf = ray.pos_from_distance(d_inf)
        
        initial_state = np.full(len(sorted_index), True)
        
        for i, j in enumerate(sorted_index):
            initial_state[i] = self[j].point_is_in(ray_pos_inf)
            
        in_states = np.full( (len(index) + 1, len(index)) , True)
        
        in_states[0] = initial_state
        
        for i in range(len(sorted_index)):
            for j in range(len(sorted_index)):
                if i == j:
                    in_states[i+1][j] = not in_states[i][j]
                else:
                    in_states[i+1][j] = in_states[i][j]
        
#         print("instate:")
#         print(in_states)
        
        which_surface = None
        
        
        for i in range(len(sorted_index)):
            if False not in in_states[i+1]:
                which_surface = i
                break
                
        if which_surface is None:
            return None
        else:
            entrance_index = sorted_index[which_surface]
            exit_index = sorted_index[which_surface + 1]
            
            if sorted_dist[which_surface] > 0:
                return self[entrance_index], surface_dist_intercept[entrance_index]
            elif sorted_dist[which_surface + 1] > 0:
                return self[exit_index], surface_dist_intercept[exit_index]
            else:
                return None
        


In [32]:
class Square(PlaneSurface):
    def __init__(self, center: Vec, length = 1, n = 1):
        
        if not isinstance(center, Vec):
            raise TypeError("center must be a Vec type")
        
        nums = [-1, 1]
        
        norms = []
        points = []
        
        for i in range(3):
            for j in nums:
                norm = np.zeros(3)
                norm[i] = j
                norms.append( Vec(norm) )
                
                point = center.vec.copy()
                point[i] = point[i] - (0.5 * length) * j
                points.append(point)
                
                
        surfaces = [Plane(Vec(norms[i]), Vec(points[i])) for i in range(6)]
                
                
        
        super().__init__(*surfaces, n = n)




In [56]:
class Source:
    def __init__(self, *rays: list):
        self.rays = rays
        
    def __len__(self):
        return len(self.rays)
    
    def __bool__(self):
        return len(self) > 0
        
    def __getitem__(self, i):
        return self.rays[i]
    
    def __setitem__(self, i, val):
        self.rays[i] = val
        
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        
        if self.index < len(self):
            val = self.rays[self.index]
            self.index += 1
            return val
        raise StopIteration
        
    

In [57]:
class Scene:
    def __init__(self, *items):
        self.items = items
        
    def __len__(self):
        return len(self.items)
    
    def __bool__(self):
        return len(self) > 0
        
    def __getitem__(self, i):
        return self.items[i]
    
    def __setitem__(self, i, val):
        self.items[i] = val
        
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        
        if self.index < len(self):
            val = self[self.index]
            self.index += 1
            return val
        raise StopIteration
        
    def update_ray(self, ray: Ray, to_level = 10):
        
        rays_traced_count = 0
        
        child_rays = [ray]
        
        if not isinstance(ray, Ray):
            raise TypeError("ray must be a Ray type")
            
        while len(child_rays) > 0:
            test_ray = child_rays.pop()        
            nearest_dist = None

            for item in self:

                intercept_info = item.find_intercept(test_ray)

                if intercept_info is not None:

                    if nearest_dist is None or intercept_info[1] < nearest_dist:
                        nearest_dist = intercept_info[1]
                        test_ray.end_boundary = intercept_info[0]

            test_ray.update_ray()
            rays_traced_count+=1
            
            if test_ray.refracted_ray is not None and test_ray.refracted_ray.level <= to_level:
                child_rays.append(test_ray.refracted_ray)
            if test_ray.reflected_ray is not None and test_ray.reflected_ray.level <= to_level:
                child_rays.append(test_ray.reflected_ray)
        
        return rays_traced_count
        
    
    def update_source(self, source: Source, **kwargs):
        
        tot_count = 0
        
        for ray in source:
            tot_count+=self.update_ray(ray, **kwargs)
        
        print(tot_count)
        return tot_count
        
        

In [58]:
rays = Source(
    *[
        Ray(
            direction = Vec(np.array([1 + i/1000, 1 + (i + 10)/1000, 1 + (i + 20)/1000])),
            start_pos = Vec(np.array([-2, -2, -2]))
        )
        for i in range(1000)
    ]
)

In [59]:
sq = Square(center = Vec(np.array([0, 0, 0])), length = 1, n = 2)
ray = Ray(direction = Vec(np.array([1.1, 1.2, 1.3])), start_pos = Vec(np.array([-2, -2, -2])))
scene = Scene(sq)



In [61]:
scene.update_source(rays)

21000


21000

In [37]:
print(ray)


class: Ray
direction: [0.5280169 0.5760185 0.62402  ]
start_pos: [-2. -2. -2.]
end_pos: [-0.5        -0.36363614 -0.22727275]
start_boundary: None
end_boundary: Plane: (norm = [1. 0. 0.], p0 = [-0.5  0.   0. ])
level: 0
refracted_ray: 
-class: Ray
-direction: [0.9053731  0.28800926 0.31201   ]
-start_pos: [-0.49999472 -0.36363038 -0.2272665 ]
-end_pos: [ 0.5        -0.04552093  0.11735205]
-start_boundary: Plane: (norm = [1. 0. 0.], p0 = [-0.5  0.   0. ])
-end_boundary: Plane: (norm = [-1.  0.  0.], p0 = [0.5 0.  0. ])
-level: 1
-refracted_ray: 
--class: Ray
--direction: [0.9053731  0.28800926 0.31201   ]
--start_pos: [ 0.50000906 -0.04551805  0.11735518]
--end_pos: None
--start_boundary: Plane: (norm = [-1.  0.  0.], p0 = [0.5 0.  0. ])
--end_boundary: None
--level: 2
--refracted_ray: None
--reflected_ray: None

-reflected_ray: 
--class: Ray
--direction: [0.67299634 0.52002674 0.5259736 ]
--start_pos: [ 0.49999094 -0.04552381  0.11734893]
--end_pos: [ 0.5        -0.04551681  0.117356